In [1]:
f = open("data.csv", 'r')


In [2]:
rows = f.readlines()
data = []

In [3]:
for row in rows:
    data.append(row.split(','))

In [4]:
data[1]

['Call Out',
 'AA2765',
 '20180102 141000',
 '[ECP] 去電確認開戶文件 客戶說後來去買美元保單就不開戶了',
 '\n']

In [5]:
import datetime
import time
def find_time_range(time_list):
    min_time = datetime.datetime(2020, 1, 1, 12, 0 ,0)
    max_time = datetime.datetime(2000, 1, 1, 12, 0, 0)
    for cur_time in time_list:
        cur_datetime = datetime.datetime.strptime(cur_time, "%Y%m%d %H%M%S")
        if cur_datetime < min_time:
            min_time = cur_datetime
        if cur_datetime > max_time:
            max_time = cur_datetime
    return min_time, max_time

In [6]:
time_list = [t[2] for t in data if t[2] != '資料日期']
# time_list

In [7]:
min_time, max_time = find_time_range(time_list)

In [8]:
min_time, max_time

(datetime.datetime(2018, 1, 2, 14, 10), datetime.datetime(2018, 12, 28, 15, 8))

In [39]:
time_range = int((max_time - min_time).total_seconds())
time_block_num = 100
time_delta = int(time_range / time_block_num)
time_group = [min_time + datetime.timedelta(seconds=time_delta * k) for k in range(time_block_num)]

PhoneCallRecordSortByTime = [[] for i in range(100)]
for idx in range(len(time_list)):
    for idy in range(1, len(time_group)):
        if datetime.datetime.strptime(time_list[idx], "%Y%m%d %H%M%S") < time_group[idy]:
            PhoneCallRecordSortByTime[idy-1].append(data[idx][3].strip())
            break
        elif idy == len(time_group) - 1:
            PhoneCallRecordSortByTime[idy].append(data[idx][3].strip())
            

In [40]:
with open('stopwords.txt', 'r', encoding='UTF-8') as f:
    stopwordList = []
    for stopword in f.readlines():
        stopwordList.append(stopword.strip())
# stopwordList[:10]

In [42]:
import jieba
import jieba.posseg as pseg
# First do some TDM without NER only using jieba and stopwords
wanted_nature = ['ns', 'nt', 'nz', 'vd', 'vn'] # remove 'n', 'a', 'v', 'nr'

PhoneCallSegments = [[] for i in range(100)]

for idx in range(len(PhoneCallRecordSortByTime)):
    for record in PhoneCallRecordSortByTime[idx]:
        segment = pseg.cut(record)
        for term, nature in segment:
            if term != ' ' and term not in stopwordList and nature in wanted_nature:
                #print(term, nature)
                PhoneCallSegments[idx].append(term)


In [43]:
import pandas as pd

def MakeWordMatrix(doc):
    word_matrix = {}
    for term in doc:
        if not (term in word_matrix.keys()):
            word_matrix[term] = 1
        else:
            word_matrix[term] += 1
    return word_matrix

TDM = None
for idx in range(len(PhoneCallSegments)):
    WordMatrix = MakeWordMatrix(PhoneCallSegments[idx])
    tmp_df = pd.DataFrame(data=WordMatrix, index=[0])    
    TDM = pd.concat([TDM, tmp_df], ignore_index=True)
    
TDM = TDM.fillna(0)
TDM

/usr/local/lib/python3.5/site-packages/ipykernel/__main__.py:16: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



,中,中信,中債會,中國,中巴,中東,中機會,中華,中銀,亞洲,...,通訊,運用,鄭,野美,銷,銷售,電詢,非,非美國,預算
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [44]:
import numpy as np
TDMArray = TDM.values
TDMArray 

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [45]:
CoOccurenceMatrix = np.dot(TDMArray.T, TDMArray)
column_name = TDM.columns.values
CoOccurenceDataFrame = pd.DataFrame(data=CoOccurenceMatrix,columns=column_name, index=column_name)
CoOccurenceDataFrame

,中,中信,中債會,中國,中巴,中東,中機會,中華,中銀,亞洲,...,通訊,運用,鄭,野美,銷,銷售,電詢,非,非美國,預算
中,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
中信,0.0,149.0,0.0,35.0,0.0,0.0,0.0,0.0,0.0,6.0,...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
中債會,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
中國,1.0,35.0,1.0,155.0,0.0,3.0,1.0,0.0,0.0,0.0,...,4.0,0.0,0.0,1.0,0.0,8.0,0.0,3.0,0.0,0.0
中巴,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
中東,0.0,0.0,0.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
中機會,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
中華,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
中銀,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
亞洲,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [46]:
CoOccurenceDataFrame.to_csv('CoOccurenceMatrix.csv')
# visualize by Gephi